In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#ROOT_DIR = "/content/drive/MyDrive/CASGEM-Egitim/Egitim-Part1/Day3-FeatureSelection/notebooks"
ROOT_DIR = "https://media.githubusercontent.com/media/yapay-ogrenme/casgem-eu-project-training-on-data-mining-2nd/main/PART1/Day3-FeatureSelection/notebooks"
DATASET_PATH = ROOT_DIR + "/datasets/"

# GİRİŞ #

Bir miktar potansiyele sahip bir dizi özellik belirledikten sonra, bunları geliştirmeye başlamanın zamanı geldi. Bu derste, tamamen Pandas ile yapabileceğiniz bir dizi yaygın dönüşümü öğreneceksiniz.

Bu derste çeşitli özellik türlerine sahip dört veri kümesi kullanacağız: 
[US Traffic Accidents](https://www.kaggle.com/sobhanmoosavi/us-accidents),
[1985 Automobiles](https://www.kaggle.com/toramky/automobile-dataset),
[Beton Formülasyonları](https://www.kaggle.com/sinamhd9/concrete-comprehensive-strength) ve 
[Müşteri Yaşam Boyu Değeri](https://www.kaggle.com/pankajjsh06/ibm-watson-marketing-customer-value-data)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)

accidents = pd.read_csv(DATASET_PATH+"accidents.csv")
autos = pd.read_csv(DATASET_PATH+"autos.csv")
concrete = pd.read_csv(DATASET_PATH+"concrete.csv")
customer = pd.read_csv(DATASET_PATH+"customer.csv")

<blockquote style="margin-right:auto; margin-left:auto; background-color: #ebf9ff; padding: 1em; margin:24px;">
<strong>Yeni Özellikleri Keşfetmeye İlişkin İpuçları</strong>
<ul>
<li>Özellikleri anlayın. Varsa, veri kümenizin <em>veri belgelerine</em> bakın.
<li> Alan bilgisi edinmek için <strong>problem alanını</strong> araştırın.Sorununuz ev fiyatlarını tahmin etmekse, örneğin biraz emlak araştırması yapın. 
Wikipedia iyi bir başlangıç noktası olabilir, ancak kitaplar ve  <a href="https://scholar.google.com/"> dergi makaleleri </a>  genellikle en iyi bilgiye sahip olacaktır.


<li>Veri görselleştirmeyi kullanın. Görselleştirme, bir özelliğin dağılımındaki patolojileri veya basitleştirilebilecek karmaşık ilişkileri ortaya çıkarabilir. Özellik mühendisliği sürecinde çalışırken veri kümenizi görselleştirdiğinizden emin olun.
<ul>
</blockquote>

# Matematiksel Dönüşümler #

Sayısal özellikler arasındaki ilişkiler, genellikle, alan araştırmanızın bir parçası olarak sıklıkla karşılaşacağınız matematiksel formüllerle ifade edilir. Pandas'da, sütunlara tıpkı sıradan sayılarmış gibi aritmetik işlemler uygulayabilirsiniz.

*Automobile* veri kümesinde bir otomobilin motorunu tanımlayan özellikler bulunur. Araştırma, potansiyel olarak yararlı yeni özellikler oluşturmak için çeşitli formüller sağlar. Örneğin strok oranı ("stroke ratio"), bir motorun ne kadar verimli olduğuna karşı ne kadar performans gösterdiğinin bir ölçüsüdür:


In [ ]:
autos["stroke_ratio"] = autos.stroke / autos.bore

autos[["stroke", "bore", "stroke_ratio"]].head()

Bir kombinasyon ne kadar karmaşıksa, bir motorun gücünün bir ölçüsü olan bu yer değiştirme ("displacement") formülü gibi bir modelin öğrenmesi o kadar zor olacaktır:


In [ ]:
autos["displacement"] = (
    np.pi * ((0.5 * autos.bore) ** 2) * autos.stroke * autos.num_of_cylinders
)

In [ ]:
autos

Veri görselleştirme, genellikle bir özelliğin güçler veya logaritmalar aracılığıyla "yeniden şekillendirilmesi" olan dönüşümler önerebilir. Örneğin, ABD Kazalarında (*US Accidents*) Rüzgar Hızı'nın (`WindSpeed`) dağılımı oldukça çarpıktır. Bu durumda logaritma onu normalleştirmede etkilidir:


In [ ]:
# If the feature has 0.0 values, use np.log1p (log(1+x)) instead of np.log
accidents["LogWindSpeed"] = accidents.WindSpeed.apply(np.log1p)

# Plot a comparison
fig, axs = plt.subplots(1, 2, figsize=(8, 4))
sns.kdeplot(accidents.WindSpeed, shade=True, ax=axs[0])
sns.kdeplot(accidents.LogWindSpeed, shade=True, ax=axs[1]);

# Özellikleri Oluşturma ve Bozma #

Genellikle daha basit parçalara faydalı bir şekilde bölünebilecek karmaşık dizelere sahip olursunuz. Bazı yaygın örnekler:

- Kimlik numaraları: ''123-45-6789''
- Telefon numaraları: `'(999) 555-0123''
- Sokak adresleri: `'8241 Kaggle Ln., Goose City, NV''
- İnternet adresleri: `'http://www.kaggle.com'
- Ürün kodları: `'0 36000 29145 2''
- Tarihler ve saatler: `'Pts 30 Eylül 07:06:05 2013''

Bunun gibi özellikler genellikle kullanabileceğiniz bir tür yapıya sahip olmalıdır. Örneğin ABD telefon numaralarında, size arayanın yerini söyleyen bir alan kodu (`'(999)'` kısmı) bulunur. 

`str` erişimcisi, `split` gibi string yöntemlerini doğrudan sütunlara uygulamanıza izin verir. *Müşteri Yaşam Boyu Değeri* (*Customer Lifetime Value*) veri seti, bir sigorta şirketinin müşterilerini tanımlayan özellikleri içerir. `Policy` özelliğinden, `Type`ı `Level`dan ayırabiliriz:


In [ ]:
customer[["Type", "Level"]] = (  # Create two new features
    customer["Policy"]           # from the Policy feature
    .str                         # through the string accessor
    .split(" ", expand=True)     # by splitting on " "
                                 # and expanding the result into separate columns
)

customer[["Policy", "Type", "Level"]].head(10)

Kombinasyonda bir etkileşim olduğuna inanmak için nedeniniz varsa, basit özellikleri oluşturulmuş bir özellikte de birleştirebilirsiniz:

In [ ]:
autos["make_and_style"] = autos["make"] + "_" + autos["body_style"]
autos[["make", "body_style", "make_and_style"]].head()

# Grup Dönüşümleri #

Son olarak, bazı kategorilere göre gruplandırılmış birden çok satırda bilgi toplayan **Grup dönüşümleri** var. Grup dönüşümüyle, "kişinin ikamet ettiği eyaletin ortalama geliri" veya "türe göre hafta içi vizyona giren filmlerin oranı" gibi özellikler oluşturabilirsiniz. Bir kategori etkileşimi keşfettiyseniz, bu kategori üzerinde bir grup dönüşümü araştırmak için iyi bir şey olabilir.

Bir toplama işlevi (aggregation function) kullanan bir grup dönüşümü, iki özelliği birleştirir: gruplamayı sağlayan kategorik bir özellik ve değerlerini toplamak istediğiniz başka bir özellik. "Devlete göre ortalama gelir" için, gruplama özelliği için "Devlet"i (`State`), toplama işlevi için "ortalama"yı (`mean`) ve toplu özellik için "Gelir"i (`Income`) seçersiniz. Bunu Pandalarda hesaplamak için `groupby` ve `transform` yöntemlerini kullanırız:



In [ ]:
customer["AverageIncome"] = (
    customer.groupby("State")  # for each state
    ["Income"]                 # select the income
    .transform("mean")         # and compute its mean
)

customer[["State", "Income", "AverageIncome"]].head(10)

`mean` işlevi, yerleşik bir data frame yöntemidir; bu, onu "dönüştürmek" (`transform`) için bir string olarak iletebileceğimiz anlamına gelir. Diğer kullanışlı yöntemler arasında `max`, `min`, `median`, `var`, `std`, ve `count` bulunur. Veri kümesinde her bir durumun meydana gelme sıklığını şu şekilde hesaplayabilirsiniz:



In [ ]:
customer["StateFreq"] = (
    customer.groupby("State")
    ["State"]
    .transform("count")
    / customer.State.count()
)

customer[["State", "StateFreq"]].head(10)

Kategorik bir özellik için bir "frekans kodlaması" oluşturmak için bunun gibi bir dönüşüm kullanabilirsiniz.

Eğitim ve doğrulama (validation) bölmelerini kullanıyorsanız, bağımsızlıklarını korumak için yalnızca eğitim kümesini kullanarak gruplandırılmış bir özellik oluşturmak ve ardından bunu doğrulama kümesine katılmak en iyisidir. Eğitim setinde `drop_duplicates` ile benzersiz bir değer kümesi oluşturduktan sonra doğrulama kümesinin `merge` yöntemini kullanabiliriz:



In [ ]:
# Create splits
df_train = customer.sample(frac=0.5)
df_valid = customer.drop(df_train.index)

# Create the average claim amount by coverage type, on the training set
df_train["AverageClaim"] = df_train.groupby("Coverage")["ClaimAmount"].transform("mean")

# Merge the values into the validation set
df_valid = df_valid.merge(
    df_train[["Coverage", "AverageClaim"]].drop_duplicates(),
    on="Coverage",
    how="left",
)

df_valid[["Coverage", "AverageClaim"]].head(10)


<strong>Özellik Oluşturma İpuçları</strong><br>
Özellikler oluştururken modelinizin kendi güçlü ve zayıf yönlerini akılda tutmak iyidir. İşte bazı yönergeler:
<ul>
<li> Doğrusal modeller toplamları ve farklılıkları doğal olarak öğrenir, ancak daha karmaşık bir şey öğrenemez.
<li> Oranları öğrenmek çoğu model için zor görünmektedir. Oran kombinasyonları genellikle bazı kolay performans kazanımlarına yol açar.
<li> Doğrusal modeller ve sinir ağları genellikle normalleştirilmiş özelliklerle daha iyi sonuç verir. Sinir ağları, özellikle 0'dan çok uzak olmayan değerlere ölçeklenmiş özelliklere ihtiyaç duyar. Tree-based modeller (random forest and XGBoost gibi) bazen normalleştirmeden yararlanabilir, ancak genellikle çok daha az fayda sağlar.
<li> Ağaç modelleri, hemen hemen her özellik kombinasyonunu yaklaşık olarak öğrenebilir, ancak bir kombinasyon özellikle önemli olduğunda, özellikle veriler sınırlı olduğunda, bunun açıkça oluşturulmasından faydalanabilirler.
<li> Sayımlar (Counts), özellikle ağaç modelleri için yararlıdır, çünkü bu modeller, birçok özellik arasında aynı anda bilgi toplamanın doğal bir yoluna sahip değildir.
